In [1]:
import numpy as np
from tensorflow.keras.models import load_model
# 모델 파일 load
model=load_model('best_chatbot3.h5')
import joblib
# tfidf파일 load
tfidf=joblib.load('tfidf_chatbot.pkl')
import re
from konlpy.tag import Okt
t=Okt()

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\feature_extraction\image.py:167: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int):


In [2]:
import pymongo
conn = pymongo.MongoClient() #몽고db 서버에 연결
answers=conn.baenangE.answers
# tfidf파일 load
tfidf_daily_life=joblib.load('tfidf_chatbot_daily_life.pkl') # 입력받은 문장을 벡터화 하기 위한 파일
daily_life=joblib.load('vetorized_chatbot_daily_life.pkl') # 기존 질문 문장들을 벡터화한 파일, 위에서 입력받은 문장과 유사도를 비교
# new_post 와 contents의 거리 구하는 함수 생성
import scipy as sp

def dist_raw(v1, v2):
    delta = v1 - v2
    return sp.linalg.norm(delta.toarray())

In [3]:
def baenangE_daily_life(str):
    # test text 벡터화
    # 형태소 분석 후 띄어쓰기로 구분하여 하나의 문장으로 만들기
    new_post_tokens = [t.morphs(row) for row in [str]]

    new_post_for_vectorize = []

    for content in new_post_tokens:
        sentence = ''
        for word in content:
            sentence = sentence + ' ' + word

        new_post_for_vectorize.append(sentence)

    new_post_for_vectorize
    new_post_vec = tfidf_daily_life.transform(new_post_for_vectorize)
  
    # 다른 결과를 얻을 수 있음
    best_doc = None
    best_dist = 65535
    best_i = None

    for i in range(0, 11823):
        post_vec = daily_life.getrow(i)

        # 함수호출
        d = dist_raw(post_vec, new_post_vec)

        if d<best_dist:
            best_dist = d
            best_i = i
    return answers.find_one({"daily_index":best_i}).get('A')

In [6]:
def baenangE(str):
    str2=re.sub(r'[^ ㄱ-ㅣ가-힣]+', '', str)
    str2=' '.join(t.nouns(str2))
    tfidf_str=tfidf.transform([str+" "+str2]).toarray()
    predicted=model.predict(tfidf_str)
    label=np.argmax(predicted)
    if label==0:
        return "반가워~~!"
    elif label==1:
        return "여행지를 추천해드릴게요."
    elif label==2:
        return "요즘 뜨는 여행지를 알려드릴게요."
    elif label==3:
        return baenangE_daily_life(str)

In [7]:
from tkinter import *
from tkinter import font
from tkinter import ttk
 
 
# GUI class for the chat
class GUI:
    
    # constructor method
    def __init__(self):
 
        # chat window which is currently hidden
        self.Window = Tk()
        self.Window.withdraw()
 
        # login window
        self.login = Toplevel()
        # set the title
        self.login.title("Login")
        self.login.resizable(width=False,
                             height=False)
        self.login.configure(width=400,
                             height=300)
        # create a Label
        self.pls = Label(self.login,
                         text="이름을 입력해주세요.",
                         justify=CENTER,
                         font="Helvetica 14 bold")
 
        self.pls.place(relheight=0.15,
                       relx=0.2,
                       rely=0.07)
        # create a Label
        self.labelName = Label(self.login,
                               text="",
                               font="Helvetica 12")
 
        self.labelName.place(relheight=0.2,
                             relx=0.1,
                             rely=0.2)
 
        # create a entry box for
        # tyoing the message
        self.entryName = Entry(self.login,
                               font="Helvetica 14")
 
        self.entryName.place(relwidth=0.4,
                             relheight=0.12,
                             relx=0.23,
                             rely=0.2)
 
        # set the focus of the cursor
        self.entryName.focus()
 
        # create a Continue Button
        # along with action
        self.go = Button(self.login,
                         text="입장",
                         font="Helvetica 14 bold",
                         command=lambda: self.goAhead(self.entryName.get()))
 
        self.go.place(relx=0.4,
                      rely=0.55)
        
        def callback(event):
            self.buttonMsg.invoke() # 클릭 동작
        self.Window.bind('<Return>', callback)
        self.Window.mainloop()
 
    def goAhead(self, name):
        self.login.destroy()
        self.layout(name)
 
    # The main layout of the chat
    def layout(self, name):
 
        self.name = name
        # to show chat window
        self.Window.deiconify()
        self.Window.title("CHATROOM")
        self.Window.resizable(width=False,
                              height=False)
        self.Window.configure(width=470,
                              height=550,
                              bg="#17202A")
        self.labelHead = Label(self.Window,
                               bg="#17202A",
                               fg="#EAECEE",
                               text=self.name,
                               font="Helvetica 13 bold",
                               pady=5)
 
        self.labelHead.place(relwidth=1)
        self.line = Label(self.Window,
                          width=450,
                          bg="#ABB2B9")
 
        self.line.place(relwidth=1,
                        rely=0.07,
                        relheight=0.012)
 
        self.textCons = Text(self.Window,
                             width=20,
                             height=2,
                             bg="#17202A",
                             fg="#EAECEE",
                             font="Helvetica 14",
                             padx=5,
                             pady=5)
 
        self.textCons.place(relheight=0.745,
                            relwidth=1,
                            rely=0.08)
 
        self.labelBottom = Label(self.Window,
                                 bg="#ABB2B9",
                                 height=80)
 
        self.labelBottom.place(relwidth=1,
                               rely=0.825)
 
        self.entryMsg = Entry(self.labelBottom,
                              bg="#2C3E50",
                              fg="#EAECEE",
                              font="Helvetica 13")
 
        # place the given widget
        # into the gui window
        self.entryMsg.place(relwidth=0.74,
                            relheight=0.06,
                            rely=0.008,
                            relx=0.011)
 
        self.entryMsg.focus()
 
        # create a Send Button
        self.buttonMsg = Button(self.labelBottom,
                                text="전송",
                                font="Helvetica 10 bold",
                                width=20,
                                bg="#ABB2B9",
                                command=lambda: self.sendButton(self.entryMsg.get()))
 
        self.buttonMsg.place(relx=0.77,
                             rely=0.008,
                             relheight=0.06,
                             relwidth=0.22)
 
        self.textCons.config(cursor="arrow")
 
        # create a scroll bar
        scrollbar = Scrollbar(self.textCons)
 
        # place the scroll bar
        # into the gui window
        scrollbar.place(relheight=1,
                        relx=0.974)
 
        scrollbar.config(command=self.textCons.yview)
 
        self.textCons.config(state=DISABLED)
 
 
#     # function to receive messages
#     def receive(self):
#         while True:
#                 message = "반가워!"
 
            
#                 self.textCons.config(state=NORMAL)
#                 self.textCons.insert(END, message+"\n\n")
#                 self.textCons.config(state=DISABLED)
#                 self.textCons.see(END)
    
    # function to basically start the thread for sending messages
    def sendButton(self, msg):
        self.textCons.config(state=NORMAL)
        self.entryMsg.delete(0, END)
        self.textCons.insert(END, self.name+" >> "+msg+"\n\n")
        self.textCons.config(state=DISABLED)
        self.textCons.see(END)
        
        message = baenangE(msg)
        
        if (message == )
        self.textCons.config(state=NORMAL)
        self.textCons.insert(END, "배낭이 >> "+message+"\n\n")
        self.textCons.config(state=DISABLED)
        self.textCons.see(END)
 
    # function to send messages
    def sendMessage(self):
        self.textCons.config(state=DISABLED)
        while True:
            message = (f"{self.name}: {self.msg}")
            client.send(message.encode(FORMAT))
            break
 
 
# create a GUI class object
g = GUI()

SyntaxError: invalid syntax (<ipython-input-7-eb27c04efa45>, line 185)